# Setting up the Benchmark file

In [ ]:
# Read input from the benchmark file
def read_input(file_path):
    with open(file_path, 'r') as file:
        num_objects = int(file.readline().strip())
        bin_capacity = int(file.readline().strip())
        weights = [int(line.strip()) for line in file]

    return num_objects, bin_capacity, weights

# Example usage:
#file_path = 'benchmark/BPP_100_100_0.1_0.7_0.txt'
#file_path = 'benchmark/unordered_benchmark50.txt'
file_path = 'BPP_50_50_0.1_0.7_0.txt'
#file_path = '/content/drive/MyDrive/[OPTIM] TPs/TP4-ACO/benchmark50.txt'
num_objects, bin_capacity, weights = read_input(file_path)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

# ACO Implementation for Bin Packing Problem (BPP)
### Hyperparamters explained :
* **num_items:** The number of items to be packed into bins.
* **bin_capacity:** The capacity of each bin. This determines the maximum weight that a bin can hold.
* **num_ants:** The number of ants in the colony. Each ant represents a potential solution to the problem.
* **alpha:** This parameter controls the influence of the pheromone trail on the ant's decision-making process. A higher alpha value gives more weight to the pheromone trail.
* **beta:** This parameter controls the influence of the heuristic information (e.g., the remaining capacity of bins) on the ant's decision-making process. A higher beta value gives more weight to the heuristic information.
* **rho:** The pheromone evaporation rate. It determines how fast the pheromone trail evaporates over time.
* **q:** A constant value used to update the pheromone trail when an ant finds a good solution. It influences the magnitude of the pheromone update.
* **max_iter:** The maximum number of iterations or generations for which the ACO algorithm will run before terminating. It controls the convergence of the algorithm and the total computational effort.

In [4]:
# Pheromone deposition, evaporation and perception

import numpy as np
import random
import copy

class Bin:
    def __init__(self, capacity):
        self.capacity = capacity
        self.items = []

    def remaining_capacity(self):
        return self.capacity - sum(self.items)

class Item:
    def __init__(self, weight):
        self.weight = weight
        self.pheromone = 1.0  # Initialize pheromone level for each item

class Ant:
    def __init__(self, bins, items, alpha, beta):
        self.bins = bins
        self.items = items
        self.alpha = alpha
        self.beta = beta

    def select_item(self):
        probabilities = []
        total_pheromone = sum([item.pheromone for item in self.items])
        for item in self.items:
            prob = (item.pheromone / total_pheromone) ** self.alpha * (1 / item.weight) ** self.beta
            probabilities.append(prob)
        probabilities = np.array(probabilities) / sum(probabilities)
        selected_index = np.random.choice(len(self.items), p=probabilities)
        return selected_index

    def pack_items(self):
        while self.items:
            selected_index = self.select_item()
            selected_item = self.items.pop(selected_index)
            packed = False
            for bin in self.bins:
                if bin.remaining_capacity() >= selected_item.weight:
                    bin.items.append(selected_item.weight)
                    selected_item.pheromone += 1  # Increase pheromone level on the selected item
                    packed = True
                    break
            if not packed:
                self.bins.append(Bin(bin_capacity))  # Create a new bin and pack the item

    # def pack_items(self):
    #   while self.items:
    #       selected_index = self.select_item()
    #       print("Selected item weight:", self.items[selected_index].weight)
    #       selected_item = self.items.pop(selected_index)
    #       print("Selected item:", selected_item.weight)
    #       packed = False
    #       for i, bin in enumerate(self.bins):
    #           if bin.remaining_capacity() >= selected_item.weight:
    #               bin.items.append(selected_item.weight)
    #               selected_item.pheromone += 1  # Increase pheromone level on the selected item
    #               print("Item packed into bin", i)
    #               packed = True
    #               break
    #       if not packed:
    #           self.bins.append(Bin(bin_capacity))  # Create a new bin and pack the item
    #       print("Bins after packing:", [bin.items for bin in self.bins])
    #       print("Remaining capacity of bins:", [bin.remaining_capacity() for bin in self.bins])


class ACO_BPP:
    def __init__(self, num_items, bin_capacity, num_ants, alpha, beta, rho, q, max_iter, weights):
        self.num_items = num_items
        self.bin_capacity = bin_capacity
        self.num_ants = num_ants
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.q = q
        self.max_iter = max_iter
        self.weights = weights  # Include weights from the benchmark

    def run(self):
        pheromones = np.ones(self.num_items)  # Initialize pheromones
        best_solution = None
        best_solution_bins = None
        best_num_bins = float('inf')

        for iteration in range(self.max_iter):
            ants = [Ant([Bin(self.bin_capacity)], [Item(weight) for weight in self.weights], self.alpha, self.beta) for _ in range(self.num_ants)]

            for ant in ants:
                ant.pack_items()

            for ant in ants:
                num_bins_used = len(ant.bins)
                if num_bins_used < best_num_bins:
                    best_num_bins = num_bins_used
                    best_solution = ant
                    #best_solution_bins = [bin.items for bin in ant.bins]
                                        # Make a deep copy of the list of bins
                    best_solution_bins = [copy.deepcopy(bin.items) for bin in ant.bins]

            # Evaporate pheromones
            for item in best_solution.items:
                item.pheromone *= (1 - self.rho)
                # Alternatively, you can evaporate pheromones for all items in all ants' items lists

            # Update pheromones based on the best solution found
            for item in best_solution.items:
                item.pheromone += self.q / num_bins_used  # Update pheromones

        return best_solution_bins


# Hyperparameters
num_ants = 10
alpha = 1
beta = 1
rho = 0.1
q = 1
max_iter = 100

# Run ACO algorithm
aco = ACO_BPP(num_objects, bin_capacity, num_ants, alpha, beta, rho, q, max_iter, weights)
best_solution_bins = aco.run()

print("Optimal nb bins: ", len(best_solution_bins))
print("Best solution bins:", best_solution_bins,"\n")
cpt = 0
for i,b in enumerate(best_solution_bins):
  cpt += len(b)
  print(f"bin{i}: nb_item={len(b)}")
  if sum(b)>bin_capacity : print(f"Bin{i} exceeds the capacity {sum(b)}>{bin_capacity}")
print(f"nb_items total = {cpt}")


Optimal nb bins:  31
Best solution bins: [[24, 20, 36, 20], [41, 23, 30], [40, 60], [20, 22, 51], [63, 28], [41, 23, 35], [29, 44, 25], [30, 54], [65, 26], [47, 44], [47, 49], [56, 43], [30, 64], [53, 26, 20], [62, 25], [29, 41, 26], [35, 54], [51, 36], [31, 37, 24], [58, 26], [43, 29, 27], [48, 36], [38, 46], [36, 63], [58, 38], [65, 28], [52, 47], [37, 57], [33, 52], [60], [53]] 

bin0: nb_item=4
bin1: nb_item=3
bin2: nb_item=2
bin3: nb_item=3
bin4: nb_item=2
bin5: nb_item=3
bin6: nb_item=3
bin7: nb_item=2
bin8: nb_item=2
bin9: nb_item=2
bin10: nb_item=2
bin11: nb_item=2
bin12: nb_item=2
bin13: nb_item=3
bin14: nb_item=2
bin15: nb_item=3
bin16: nb_item=2
bin17: nb_item=2
bin18: nb_item=3
bin19: nb_item=2
bin20: nb_item=3
bin21: nb_item=2
bin22: nb_item=2
bin23: nb_item=2
bin24: nb_item=2
bin25: nb_item=2
bin26: nb_item=2
bin27: nb_item=2
bin28: nb_item=2
bin29: nb_item=1
bin30: nb_item=1
nb_items total = 70


# Automatic hyperparameter tuning
### Preprocessing phase before ACO algorithm

In [5]:
# Evaluation function
def evaluate_aco(num_items, bin_capacity, num_ants, alpha, beta, rho, q, max_iter):
    # Run ACO algorithm
    aco = ACO_BPP(num_items, bin_capacity, num_ants, alpha, beta, rho, q, max_iter, weights)
    best_solution_bins = aco.run()

    # Calculate performance metric (e.g., number of bins used)
    performance_metric = len(best_solution_bins)

    return performance_metric

In [ ]:
# Grid search
'''
 Grid search exhaustively evaluates the algorithm's performance for every combination of hyperparameters specified in a predefined grid.
 It divides the hyperparameter space into a grid and systematically tests each point in the grid.
'''
from sklearn.model_selection import ParameterGrid

def grid_search(param_grid):
    best_performance = float('inf')
    best_hyperparams = None

    param_combinations = list(ParameterGrid(param_grid))

    for params in param_combinations:
        performance = evaluate_aco(**params)

        if performance < best_performance:
            best_performance = performance
            best_hyperparams = params

    return best_hyperparams, best_performance

# Define parameter grid for grid search
param_grid = {
    'num_items': [10, 50, 100],
    'bin_capacity': [1, 5, 10],
    'num_ants': [5, 10, 20],
    'alpha': [0.1, 0.5, 1],
    'beta': [0.1, 0.5, 1],
    'rho': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    'q': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5],
    'max_iter': [100, 200, 300]

}

# Perform grid search
best_hyperparams, best_performance = grid_search(param_grid)

print("Best hyperparameters:", best_hyperparams)
print("Best performance:", best_performance)

In [ ]:
# Random search
'''
Random search randomly samples hyperparameters from predefined ranges or distributions and evaluates the algorithm's performance for each sampled combination.
It does not systematically explore the hyperparameter space but focuses on randomly selected points.
'''
def random_search(param_ranges, num_iterations):
    best_performance = float('inf')
    best_hyperparams = None

    for _ in range(num_iterations):
        hyperparams = {param: random.uniform(param_range[0], param_range[1]) for param, param_range in param_ranges.items()}
        performance = evaluate_aco(**hyperparams)

        if performance < best_performance:
            best_performance = performance
            best_hyperparams = hyperparams

    return best_hyperparams, best_performance

# Define parameter ranges for random search
param_ranges = {
    'num_items': (10, 100),
    'bin_capacity': (1, 10),
    'num_ants': (5, 20),
    'alpha': (0.1, 2),
    'beta': (0.1, 2),
    'rho': (0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9),
    'q': (0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5),
    'max_iter': (100, 500)
}

# Perform random search
best_hyperparams, best_performance = random_search(param_ranges, num_iterations=10)

print("Best hyperparameters:", best_hyperparams)
print("Best performance:", best_performance)